In [1]:
import numpy as np
from pymatreader import read_mat
import os
from torch.utils.data import Dataset
import torch
import librosa
import scipy
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
def load_subject(i):

    dir = "C:/Users/gauta/Thesis/ExJobb/ID" + str(1000+i) +"/PreprocIV_ID" +str(1000+i)+ "_mast.mat"
    temp_data = read_mat(dir)
    temp_data = temp_data['ICAcleaned_data_allsess']
    data = np.vstack([np.array(temp_data['ICAcleaned_data_allsess'][i]['trial'])[:,0:64,256*20:256*53] for i in range(len(temp_data['ICAcleaned_data_allsess']))])
    attended = ["C:/Users/gauta/Thesis/ExJobb/Files_Audio/" + temp_data['ICAcleaned_data_allsess'][k]['targetfiles'][i].split('\\')[-1] for k in range(len(temp_data['ICAcleaned_data_allsess'])) for i in range(20)]
    masker = ["C:/Users/gauta/Thesis/ExJobb/Files_Audio/" + temp_data['ICAcleaned_data_allsess'][k]['maskerfiles'][i].split('\\')[-1] for k in range(len(temp_data['ICAcleaned_data_allsess'])) for i in range(20)]
    

    return (data,attended,masker)

In [4]:
def load_subject(i):

    dir = "C:/Users/gauta/Thesis/ExJobb/ID" + str(1000+i) +"/PreprocIV_ID" +str(1000+i)+ "_mast.mat"
    temp_data = read_mat(dir)
    temp_data = temp_data['ICAcleaned_data_allsess']

    return temp_data

In [5]:
def cvt2env(dirs):
    out = []
    for dir in dirs:
        wav,sr = librosa.load(dir)
        env = np.abs(scipy.signal.hilbert(wav))
        env = scipy.signal.resample_poly(env,1024,sr)
        env = scipy.signal.decimate(env,8)
        env = scipy.signal.decimate(env,2)

        out.append(env)
    return np.array(out)


In [53]:
data = load_subject(2)

In [6]:
def cvt2dirs(dirs):
    dirs =  list(map(lambda x: x.split('\\')[-1],dirs))
    out = [a+b for a,b in zip(["C:/Users/gauta/Thesis/ExJobb/Files_Audio/"]*20,dirs)]
    return out

In [13]:
mode = ["offos","onos","offpol","onpol"]

In [19]:
for i in tqdm(range(1,33)):
    if i==14:
        continue
    trials = load_subject(i)
    for j,trial in enumerate(trials):
        EEG = scipy.signal.decimate(trial['trial'],4)
        attended = cvt2env(cvt2dirs(trial['targetfiles']))
        masker = cvt2env(cvt2dirs(trial['maskerfiles']))
        for k,(eeg,att,mas) in enumerate(zip(EEG,attended,masker)):
            np.savez("EriksholmFiles/"+mode[j]+"subject_" + str(i) + "_trial_"+ str(k + j*20),EEG=eeg,attended=att,masker=mas)

100%|██████████| 32/32 [11:17<00:00, 21.18s/it]


In [2]:
def findfiles(search_dir, prefix):
    matching_files = []

    for dirpath, _, filenames in os.walk(search_dir):
        for filename in filenames:
            if filename.startswith(prefix):
                full_path = os.path.join(dirpath, filename)
                matching_files.append(full_path)

    return matching_files

In [3]:
k = findfiles("EriksholmFiles/","off")

In [4]:
from eriksdataset import EEG_Dataset

In [5]:
data = EEG_Dataset(files=k,seconds=3,overlap=0.5)

In [6]:
p = data.__getitem__(3)

(20, 2112)
(43, 64, 192)


ValueError: negative dimensions are not allowed

In [10]:
544896/(66*64*3)

43.0

In [30]:
3*64

192